In [6]:
# ! pip install pyod -q
# ! pip install suod -q

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')
# ! cp -r /content/drive/MyDrive/Study/MIPT_magistery/qualification_work/data .

In [5]:
import os
import random
import sys
from pprint import pprint

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# for statistics
from scipy.stats import kstest

# preprocessing
from sklearn.preprocessing import StandardScaler

# ml methods
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM

#claster methods
from sklearn.cluster import DBSCAN

# pyod
# from pyod.models.ecod import ECOD
# from pyod.models.suod import SUOD
# from pyod.models.lof import LOF
# from pyod.models.copod import COPOD
# from pyod.models.iforest import IForest

# metrics
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

def seed_everything(seed):
    # фискирует максимум сидов для корректности сравнения разных экспериментов
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed(seed)
    # torch.backends.cudnn.deterministic = True
SEED = 42
seed_everything(SEED)

# Datasets

In [7]:
# Синтетически сгенерированные нормально распределенные данные
def make_norm_data(rows, columns, noise_percent=0.05):
    data_norm = pd.DataFrame(data=np.random.normal(
                            loc=0, scale=1, size=(rows, columns)))
    noise = pd.DataFrame(data=np.random.uniform(
                            low=-6, high=6, size=(int(data_norm.shape[0]*noise_percent), data_norm.shape[1])))

    data_noise = pd.DataFrame()
    for feature in noise.columns:
        filter = (
            (noise[feature] < data_norm[feature].min())
            | (noise[feature] > data_norm[feature].max())
            )
        data_noise = pd.concat([data_noise, noise[filter]])
    data_noise = data_noise.drop_duplicates()
    data_norm['anomaly'] = 0
    data_noise['anomaly'] = 1

    df_norm = pd.concat((data_norm, data_noise))
    return df_norm

df_norm = make_norm_data(1000, 8)
df_norm

,0,1,2,3,4,5,6,7,anomaly
0,0.496714,-0.138264,0.647689,1.523030,-0.234153,-0.234137,1.579213,0.767435,0
1,-0.469474,0.542560,-0.463418,-0.465730,0.241962,-1.913280,-1.724918,-0.562288,0
2,-1.012831,0.314247,-0.908024,-1.412304,1.465649,-0.225776,0.067528,-1.424748,0
3,-0.544383,0.110923,-1.150994,0.375698,-0.600639,-0.291694,-0.601707,1.852278,0
4,-0.013497,-1.057711,0.822545,-1.220844,0.208864,-1.959670,-1.328186,0.196861,0
...,...,...,...,...,...,...,...,...,...
32,0.063984,1.772509,0.031678,-0.098264,-3.938531,-2.655435,-4.093768,-3.652004,1
38,-2.800126,2.593122,-1.268901,2.124984,-0.288220,5.925277,2.746826,5.429896,1
5,2.151800,-0.353991,-1.345218,-1.488481,1.680586,-0.062468,4.189587,4.496424,1
37,-2.554173,0.008819,-0.528646,-1.866863,2.324896,3.114742,4.235808,5.674279,1


In [8]:
# SKAB data

all_files=[]
for root, dirs, files in os.walk("data/"):
    for file in files:
        if file.endswith(".csv"):
             all_files.append(os.path.join(root, file))

# формируем датафрейм
dfs=[]
for path in all_files:
    df = pd.read_csv(path,index_col='datetime',sep=';',parse_dates=True)
    # print(path, df.shape)
    dfs.append(df)
# print('Features:')
# for col in dfs[2].columns:
#     print('\t',col)
dfs = [df for df in dfs if df.shape[1] == 10]
df_skab = pd.concat(dfs)
# print(df_skab.shape)
df_skab = df_skab.drop_duplicates()
df_skab = df_skab.drop('changepoint', axis=1)
display(df_skab)

,Accelerometer1RMS,Accelerometer2RMS,Current,Pressure,Temperature,Thermocouple,Voltage,Volume Flow RateRMS,anomaly
datetime,,,,,,,,,
2020-03-09 14:34:41,0.028034,0.040930,0.823535,-0.273216,68.6704,24.5900,229.932,32.0268,0.0
2020-03-09 14:34:42,0.027804,0.040812,1.152880,0.054711,68.6931,24.5905,219.586,32.9740,0.0
2020-03-09 14:34:43,0.028303,0.041978,1.200770,-0.273216,68.6201,24.5874,227.329,32.0268,0.0
2020-03-09 14:34:44,0.027952,0.040822,0.999986,0.054711,68.5410,24.5848,222.197,32.9740,0.0
2020-03-09 14:34:45,0.028193,0.041434,0.748154,-0.273216,68.5854,24.5865,225.329,32.0000,0.0
...,...,...,...,...,...,...,...,...,...
2020-02-08 17:06:08,0.222106,0.263904,2.176580,0.382638,88.3026,29.8332,214.814,127.3190,0.0
2020-02-08 17:06:09,0.222804,0.265610,2.383720,0.054711,88.3000,29.8287,228.293,127.6830,0.0
2020-02-08 17:06:10,0.222702,0.271478,2.375500,-0.273216,88.1713,29.8370,201.892,128.0000,0.0


In [9]:
datasets = {
    'df_norm': df_norm,
    'df_skab': df_skab
}

# Статический метод выявления выбросов (3 сигма)

In [12]:
def check_kstest(df, alpha=0.01):
    columns = list(df.columns)
    cols_with_pos_res = []
    for col in columns:
        if kstest(df[col], 'norm').pvalue > alpha:
            cols_with_pos_res.append(col)
    return cols_with_pos_res

def outliers_z_score(data, feature):
    x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 3 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    print(f'Признак {feature}. Число выбросов по методу z-отклонения: {outliers.shape[0]}')
    return outliers

def z_score_method(df):
    """
    Метод работает с нормально распределенными данными,
    при чем с небольшим (до 10%) количеством выбросов.
    С ненормально распределенными данными не работает.
    """
    norm_cols = check_kstest(df)
    if norm_cols:
        print('z_score_method')
        outliers = pd.DataFrame()
        for col in norm_cols:
            outliers = pd.concat((outliers, outliers_z_score(df, col)))
        outliers = outliers.drop_duplicates()
        outliers['anomaly_preds'] = 1
        df_with_marked_outliers = df.merge(outliers, how='outer').fillna(0)
        return outliers, df_with_marked_outliers
    else:
        print('The data are not normally distributed')
        return None, df

def stat_method(X, y):
    outliers, X = z_score_method(X)
    if outliers is not None:
        print(score_metrics(y, X['anomaly_preds']))

# Метрики

In [30]:
# f1_score
def score_metrics(real_outliers, pred_outliers):
    scores = {}
    # scores['f1_score'] = f1_score(real_outliers, pred_outliers)
    scores = classification_report(real_outliers, pred_outliers)
    return scores

# ML методы

In [15]:
def preprocessing(df):
    X = df.copy()
    y = X.pop('anomaly')

    # preprocessing
    columns = list(X.columns)
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(data=X, columns=columns)
    return X, y

def ml_method(X, y, model, params):
    grid = GridSearchCV(
                estimator=model(),
                param_grid=params,
                scoring='f1_micro',
                return_train_score=True).fit(X, y)
    pred = grid.best_estimator_.predict(X)
    pred[pred != -1] = 0
    pred[pred == -1] = 1
    print(grid.best_estimator_)
    print(score_metrics(y, pred))

In [48]:
ml_models = {}
ml_models['IsolationForest'] = {
    'model': IsolationForest,
    'params': {
        'n_estimators': [50, 100, 500],
        'bootstrap': [True, False],
        }
    }
ml_models['OneClassSVM'] = {
    'model': OneClassSVM,
    'params': {
        'nu': [0.001, 0.01, 0.1, 0.5, 1],
    }
}

In [ ]:
for df_name, df in datasets.items():
    print(df_name)
    X, y = preprocessing(df)
    stat_method(X, y)
    for model in ml_models:
        ml_method(X, y, ml_models[model]['model'], 
                        ml_models[model]['params'])
    print('-' * 10)

# ECOD

In [ ]:
clf = ECOD()
clf.fit(df_norm[['x', 'y']])
pred = clf.predict(df_norm[['x', 'y']])
print(score_metrics(df_norm['anomaly'], pred))

              precision    recall  f1-score   support

           0       1.00      0.93      0.96      1000
           1       0.33      1.00      0.49        34

    accuracy                           0.93      1034
   macro avg       0.66      0.97      0.73      1034
weighted avg       0.98      0.93      0.95      1034



In [ ]:
clf = ECOD()
X, y = preprocessing(df_skab)
clf.fit(X)
pred = clf.predict(X)
print(score_metrics(y, pred))

              precision    recall  f1-score   support

         0.0       0.69      0.95      0.80     23853
         1.0       0.71      0.20      0.31     13067

    accuracy                           0.69     36920
   macro avg       0.70      0.58      0.56     36920
weighted avg       0.69      0.69      0.63     36920



In [1]:
# # initialized a group of outlier detectors for acceleration
# detector_list = [LOF(n_neighbors=15), LOF(n_neighbors=20),
#                  LOF(n_neighbors=25), LOF(n_neighbors=35),
#                  COPOD(), IForest(n_estimators=100),
#                  IForest(n_estimators=200)]

# # decide the number of parallel process, and the combination method
# # then clf can be used as any outlier detection model
# clf = SUOD(base_estimators=detector_list, n_jobs=2, combination='average',
#            verbose=False)

# X, y = preprocessing(df_skab)
# clf.fit(X)
# pred = clf.predict(X)
# print(score_metrics(y, pred))

# DBSCAN

In [ ]:
for df_name, df in datasets.items():
    print(df_name)
    X, y = preprocessing(df)
    pred = DBSCAN(eps=5, min_samples=df.shape[1]+1).fit_predict(X)
    print(set(pred))
    pred[pred != -1] = 0
    print(set(pred))
    pred[pred == -1] = 1
    print(set(pred))
    print(score_metrics(df['anomaly'], pred))

df_norm
{0, -1}
{0, -1}
{0, 1}
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1000
           1       1.00      0.58      0.73        50

    accuracy                           0.98      1050
   macro avg       0.99      0.79      0.86      1050
weighted avg       0.98      0.98      0.98      1050

df_skab
